In [2]:
import time
import pytesseract
import pyperclip
import ctypes
import tkinter as tk
from tkinter import messagebox
from PIL import ImageGrab
import win32clipboard

# 設定 Tesseract OCR 路徑
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

def notify(title, text):
    """ 顯示 Windows 通知 """
    print(f"[通知] {title}: {text}")
    ctypes.windll.user32.MessageBoxW(0, text, title, 1)

def get_clipboard_format():
    """ 取得剪貼簿格式，避免 OpenClipboard 失敗 """
    print("[INFO] 嘗試取得剪貼簿格式...")
    for attempt in range(5):  # 最多嘗試 5 次
        try:
            time.sleep(0.2)  # 避免搶占剪貼簿
            win32clipboard.OpenClipboard()
            formats = win32clipboard.EnumClipboardFormats(0)
            win32clipboard.CloseClipboard()
            print(f"[INFO] 剪貼簿格式: {formats if formats else -1}")
            return formats if formats else -1  # 若無格式則回傳 -1
        except Exception as e:
            print(f"[ERROR] 剪貼簿存取錯誤 (第 {attempt+1} 次): {e}")
            time.sleep(0.5)  # 稍作延遲再重試
    print("[ERROR] 無法存取剪貼簿，回傳 -1")
    return -1  # 若 5 次都失敗，回傳 -1

def wait_for_clipboard_change(timeout=20):
    """ 監聽剪貼簿變化，等待圖片出現在剪貼簿中 """
    print(f"[INFO] 開始監聽剪貼簿變化，超時時間 {timeout} 秒")
    start_time = time.time()
    initial_format = get_clipboard_format()  # 取得初始剪貼簿格式

    while time.time() - start_time < timeout:
        print("[INFO] 檢查剪貼簿內容...")
        current_format = get_clipboard_format()
        if current_format != initial_format:  # 偵測剪貼簿內容變化
            print("[INFO] 剪貼簿內容變化，檢查是否為圖片...")
            image = ImageGrab.grabclipboard()
            if image is not None:
                print("[INFO] 成功取得剪貼簿中的圖片！")
                return image
        time.sleep(0.5)  # 每 0.5 秒檢查一次剪貼簿

    print("[WARNING] 超過 20 秒未偵測到剪貼簿變化")
    return None  # 超時未偵測到變化

def process_screenshot():
    """ OCR 處理截圖 """
    print("[INFO] 開始處理截圖")
    screenshot = wait_for_clipboard_change()
    if screenshot is None:
        print("[ERROR] 未偵測到截圖，程式結束")
        notify("錯誤", "未偵測到截圖，程式已自動關閉！")
        return

    # 顯示彈窗詢問是否要 OCR
    print("[INFO] 顯示 OCR 確認彈窗")
    root = tk.Tk()
    root.withdraw()  # 隱藏主視窗
    if not messagebox.askyesno("OCR 辨識", "是否要進行 OCR 並存入剪貼簿？"):
        print("[INFO] 使用者取消 OCR")
        notify("已取消", "OCR 辨識已取消。")
        return

    # OCR 辨識
    print("[INFO] 開始進行 OCR 文字辨識")
    text = pytesseract.image_to_string(screenshot, lang="chi_tra+eng")
    if text.strip():
        print("[INFO] OCR 辨識成功，將文字存入剪貼簿")
        pyperclip.copy(text)
        notify("OCR 完成", "已將辨識內容儲存至剪貼簿！")
        print(f"[OCR 結果]\n{text}")
    else:
        print("[ERROR] OCR 辨識失敗，未能辨識出文字")
        notify("OCR 失敗", "未能辨識出文字，請重試。")

process_screenshot()


[INFO] 開始處理截圖
[INFO] 開始監聽剪貼簿變化，超時時間 20 秒
[INFO] 嘗試取得剪貼簿格式...
[INFO] 剪貼簿格式: 49385
[INFO] 檢查剪貼簿內容...
[INFO] 嘗試取得剪貼簿格式...
[INFO] 剪貼簿格式: 49385
[INFO] 檢查剪貼簿內容...
[INFO] 嘗試取得剪貼簿格式...
[INFO] 剪貼簿格式: 49385
[INFO] 檢查剪貼簿內容...
[INFO] 嘗試取得剪貼簿格式...
[INFO] 剪貼簿格式: 49385
[INFO] 檢查剪貼簿內容...
[INFO] 嘗試取得剪貼簿格式...
[INFO] 剪貼簿格式: 49385
[INFO] 檢查剪貼簿內容...
[INFO] 嘗試取得剪貼簿格式...
[INFO] 剪貼簿格式: 49385
[INFO] 檢查剪貼簿內容...
[INFO] 嘗試取得剪貼簿格式...
[INFO] 剪貼簿格式: 49385
[INFO] 檢查剪貼簿內容...
[INFO] 嘗試取得剪貼簿格式...
[INFO] 剪貼簿格式: 49385
[INFO] 檢查剪貼簿內容...
[INFO] 嘗試取得剪貼簿格式...
[INFO] 剪貼簿格式: 49385
[INFO] 檢查剪貼簿內容...
[INFO] 嘗試取得剪貼簿格式...
[INFO] 剪貼簿格式: 49385
[INFO] 檢查剪貼簿內容...
[INFO] 嘗試取得剪貼簿格式...
[INFO] 剪貼簿格式: 49161
[INFO] 剪貼簿內容變化，檢查是否為圖片...
[INFO] 成功取得剪貼簿中的圖片！
[INFO] 顯示 OCR 確認彈窗
[INFO] 開始進行 OCR 文字辨識
[INFO] OCR 辨識成功，將文字存入剪貼簿
[通知] OCR 完成: 已將辨識內容儲存至剪貼簿！
[OCR 結果]
# OCR Saat
print("[INFO] BAY8#=T OCR SCS")
text = pytesseract.image_to_string(screenshot, lang="chi_tra+eng")
if text.strip():
print("[INFO] OCR }asiAel) » SCA STRASS")
pyperclip.copy(text)
no